Team members: Jeevan Rai & Arman Kamal

In [2]:
!pip install tensorflow==2.16.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.7/590.7 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 82.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: ml-dtypes
    Fo

In [ ]:
!pip install tqdm==4.66.3

Need at least numpy 1.26 for tensorflow 2.16.2 (restart the kernel)

In [ ]:
!pip install numpy==1.26

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 68.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.16.2 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.16.2 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.6 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.16.2 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.0 which is incompat

In [ ]:
!pip install tf-keras==2.16

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.0 MB/s eta 0:00:00
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.18.0
    Uninstalling tf_keras-2.18.0:
      Successfully uninstalled tf_keras-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tf-keras>=2.18.0, but you have tf-keras 2.16.0 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.16.2 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tf-keras~=2.17, but you have tf-keras 2.16.0 which is incompatible.


In [ ]:
#!pip install matplotlib==3.5.0

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import torch
import numpy as np
import tensorflow as tf
import tf_keras
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
#from torch.utils.data import random_split
#from torch.utils.data import TensorDataset
from torch.utils.data import Dataset
from torch.optim import Adam
import torch.optim.lr_scheduler as lr_scheduler  # to adjust learning rate dynamically during training
from sklearn.metrics import accuracy_score
from torch import nn
from tqdm.notebook import tqdm, trange
#import matplotlib
#import matplotlib.pyplot as plt
import time
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# Check versions
print(torch.__version__)
print(tf.__version__)
print(tf_keras.__version__)
print(np.__version__)
#print(matplotlib.__version__)

2.6.0+cu124
2.16.2
2.16.0
1.26.0


# Prepare the dataset

In [ ]:
# Load MS MARCO v1.1 (passage ranking version)
dataset = load_dataset("microsoft/ms_marco", "v1.1")

In [ ]:
# Let's look at the dataset structure
dataset

DatasetDict({
    validation: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 10047
    })
    train: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 82326
    })
    test: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 9650
    })
})

In [ ]:
# Count rows in each split
for split in dataset:
    print(f"{split.capitalize()} Set: {len(dataset[split])} rows")

Validation Set: 10047 rows
Train Set: 82326 rows
Test Set: 9650 rows


Here are the number of samples for validation, training, and testing.

In [ ]:
# Convert each datasets into separate panda dataframe
train_df = pd.DataFrame(dataset["train"])
validation_df = pd.DataFrame(dataset["validation"])
test_df = pd.DataFrame(dataset["test"])

In [ ]:
# Remove unnecessary feature columns from each datasets
train_df = train_df.drop(columns=["answers", "passages", "query_id", "wellFormedAnswers"])
test_df = test_df.drop(columns=["answers", "passages", "query_id", "wellFormedAnswers"])
validation_df = validation_df.drop(columns=["answers", "passages", "query_id", "wellFormedAnswers"])

In [ ]:
# Save the datasets to a CSV file to avoid future downloads
train_df.to_csv("train_df.csv", index=False)
validation_df.to_csv("validation.csv", index=False)
test_df.to_csv("test_df.csv", index=False)

In [ ]:
# Load the saved datasets and convert them to pandas dataframe
train_df = pd.read_csv("train_df.csv")
validation_df = pd.read_csv("validation.csv")
test_df = pd.read_csv("test_df.csv")

In [ ]:
# Check for missing values
train_df.isnull().sum()
validation_df.isnull().sum()
test_df.isnull().sum()

,0
query,0
query_type,0


In [ ]:
# Check to ensure all datasets have all 5 labels
train_df["query_type"].nunique()
validation_df["query_type"].nunique()
test_df["query_type"].nunique()

5

In [ ]:
# Encode the labels

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'query_type' column and store the result in a new column 'query_type_encoded'
train_df['query_type_encoded'] = label_encoder.fit_transform(train_df['query_type'])
validation_df['query_type_encoded'] = label_encoder.fit_transform(validation_df['query_type'])
test_df['query_type_encoded'] = label_encoder.fit_transform(test_df['query_type'])

In [ ]:
# encoded labels
print (dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


{'description': 0, 'entity': 1, 'location': 2, 'numeric': 3, 'person': 4}


In [ ]:
# Assign feature and target column

# TRAIN
X_train = train_df["query"]
y_train = train_df["query_type_encoded"]

# VALIDATION
X_val = validation_df["query"]
y_val = validation_df["query_type_encoded"]

# TEST
X_test = test_df["query"]
y_test = test_df["query_type_encoded"]

# Baseline model

We will load the pretrained BERT uncased model. We want to perform classification, so we will need to get BertForSequenceClassifiction as it includes a fully connected classification layer on top of the BERT model

In [ ]:
# Load the BERT model
model_base = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=5,
                                                      output_hidden_states=False
                                                     )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Let's evaluate the base model architecture
model_base

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

**Base model architecture:**


*   We have BERT model that contains embeddings, word_embeddings, position_embeddings, token_type_embeddings, LayerNorm, and dropout
*   It then goes into BertEncoder that is simply 12 identical layers where we have self attention layer, query, key, value, and dropout

*   Then it goes through couple of dense layers: output "BertSelfOutput, intermediate "BertIntermediate", and output "BertOutput
*   Then it goes through a pooling layer "BertPooler" that has output of 768 (linear)

*   Finally it goes through a classifier layer (untrained) that expects intput of 768 and will have output of 20 (number of layers for classification)













In [ ]:
# place the based model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_base.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Tokenization

In [ ]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Tokenize the queries in train_df
train_encodings = tokenizer(
    train_df['query'].tolist(),       # Input text
    truncation=True,                  # Truncate to max length
    padding=True,                     # Pad to max length
    max_length=128,                   # Optional: set max sequence length
    return_tensors="pt"               # Return PyTorch tensors
)

In [ ]:
print(train_encodings.keys())
print(train_encodings['input_ids'].shape)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([82326, 52])


We can see that for our training set, we have 82,326 queries that have been tokenized into a sequence of 52 tokens.

In [ ]:
# Tokenize the queries in validation_df
validation_encodings = tokenizer(
    validation_df['query'].tolist(),       # Input text
    truncation=True,                  # Truncate to max length
    padding=True,                     # Pad to max length
    max_length=128,                   # Optional: set max sequence length
    return_tensors="pt"               # Return PyTorch tensors
)

# Tokenize the queries in test_df
test_encodings = tokenizer(
    test_df['query'].tolist(),       # Input text
    truncation=True,                  # Truncate to max length
    padding=True,                     # Pad to max length
    max_length=128,                   # Optional: set max sequence length
    return_tensors="pt"               # Return PyTorch tensors
)

In [ ]:
print(validation_encodings.keys())
print(validation_encodings['input_ids'].shape)

print(test_encodings.keys())
print(test_encodings['input_ids'].shape)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([10047, 34])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([9650, 28])


## Tune model for training

Since BERT models expect input in a specific format: batches of input_ids, attention_mask, and labels, all as tensors, we will wrap these inputs in a Dataset class.

In [ ]:
train_df.head()

,query,query_type,query_type_encoded
0,what is rba,description,0
1,was ronald reagan a democrat,description,0
2,how long do you need for sydney and surroundin...,numeric,3
3,price to install tile in shower,numeric,3
4,why conversion observed in body,description,0


In [ ]:
# Create PyTorch custom dataset class to build train, test, and validation datasets
class QueryDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings  #dict returned by tokenizer
        self.labels = labels  # list of classification labels

    # To allow PyTorch to fetch a single samply by index (what BERT expects)
    def __getitem__(self, idx):

        # get input_ids & attention_mask for one item. convert label to Pytorch tensors
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'token_type_ids': self.encodings['token_type_ids'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    # to return total number of samples
    def __len__(self):
        return len(self.labels)

Now using the class above, let's create the three datasets. Each gets tokenized inputs (encodings) and a list of labels.

In [ ]:
train_dataset = QueryDataset(train_encodings, y_train.tolist())
val_dataset = QueryDataset(validation_encodings, y_val.tolist())
test_dataset = QueryDataset(test_encodings, y_test.tolist())

In [ ]:
print(train_df.head(1))
train_dataset[0]

         query   query_type  query_type_encoded
0  what is rba  description                   0


{'input_ids': tensor([  101,  2054,  2003, 21144,  2050,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]),
 'labels': tensor(0)}

This is a quick check for first sample in train dataset. The dictionary is good for training the model.  

Now, let's create data loaders for each dataset. The batch size is 16. To maintain randomness, we will shuffle the samples.

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

Set the hyperparameters and other parameters for training.

In [ ]:
# training iterations
epochs = 4

# define the optimizer
optimizer = Adam(model_base.parameters(), lr=2e-5)

Batch_size = 16
train_size = len(train_df)
val_size = len(validation_df)

In [1]:
# setup tracking for training
history_base = {}  # empty dict to store training history of the base model
history_base['val_loss'] = []  # stores validation loss values
history_base['train_loss'] = []  # stores training loss values

# setting up a progress bar for epochs
with trange(epochs, desc="Epoch") as pbar:

    for epoch in pbar:
        model_base.train()  # put the model in training mode
        total_loss  = 0.0  # track the training loss

        # iterate over training data in batchs using train_dataloader.
        for batch in tqdm(train_dataloader, desc=f"Training {epoch}", total=len(train_dataloader), leave=False):

            # send batch (tuple of below input parameters & labels) to GPU and separate for the model
            #batch = tuple(s.to(device) for s in batch)
            batch = {k: v.to(device) for k, v in batch.items()}   # Send all tensor values to GPU
            input_ids, input_mask, input_types, labels = batch  # unpack those parameters of the tuple

            # pass these parameters thru model to perform forward pass. This will compute model's output (includes training loss)
            #output = model_base(input_ids=input_ids, attention_mask=input_mask, labels=labels)
            output = model_base(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                token_type_ids=batch['token_type_ids'],
                labels=batch['labels']
            )

            # back propagation
            output.loss.backward() # computes gradients by backpropagation using the loss
            optimizer.step() # use gradients in optimizer to update model's parameters
            optimizer.zero_grad() # clear out gradients of optimized tensors to avoid accumulation of gradients between different batches

            # keep track of total train loss. Need to multiply loss per sample by batch size for total loss
            #total_loss += output.loss.item() * Batch_size
            total_loss += output.loss.item() * batch['input_ids'].size(0)

        # calcualte the avg. training loss and add it to the train_loss
        total_loss  = total_loss / train_size
        history_base['train_loss'].append(total_loss)


        # print the train loss in the progress bar for the epochs
        pbar.set_postfix(loss=f"{total_loss:.2f}")

        # at end of epoch, report the evaluation
        model_base.eval()  # Set model to evaluation mode (droput layers and other training specific operations are turned off)
        total_loss  = 0.0  #rest the total loss to 0 to track the validation loss

        # turn of gradient calculation as we're only evaluating the model
        with torch.no_grad():

            # evaluation in batches (same as above)
            for batch in tqdm(val_dataloader, desc=f"Evaluating {epoch}", total = len(val_dataloader), leave=False):
                #batch = tuple(s.to(device) for s in batch) # send to GPU
                batch = {k: v.to(device) for k, v in batch.items()}

                output = model_base(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    token_type_ids=batch['token_type_ids'],
                    labels=batch['labels']
                )

                total_loss += output.loss.item() * batch['input_ids'].size(0)
                #input_ids, input_mask, input_types, labels = batch
                #output = model_base(input_ids, input_types, input_mask, labels=labels)
                #total_loss += output.loss.item() * Batch_size

            total_loss  = total_loss / val_size
            history_base['val_loss'].append(total_loss)

        # print the eval loss in the progress bar for the epochs
        pbar.set_postfix(loss=f"{total_loss:.2f}")

NameError: name 'trange' is not defined

In [ ]:
# Save the trained model & tokenizer (for google colab)
model_base.save_pretrained("bert_model_base")
tokenizer.save_pretrained('/content/bert_model_base')

## Train base model

# Tune the model to make classification

# Model Evaluation (Baseline)

# Injection of Noise

# Model Evaluation & compare against baseline

# Customize the model for further exploration

# Model Evaluation & compare against baseline